# TP intersections

L'objectif de ce TP est de créer un simulateur de trafic routier simple, avec intersections. Ce SMA contiendra plusieurs types d'agents : les véhicules et les intersections. L'objectif est de fluidifier le trafic le plus possible.

In [1]:
import random

# I. Implémentation du simulateur

Dans cette partie, nous implémentons un simulateur de trafic routier. Ce simulateur a pour vocation d'être le plus simple possible : nous suivons le paradigme KISS (*Keep It Simple, Stupid*). Le réseau routier est composé de routes sur lesquelles circulent des voitures. Des intersections sont implémentées au bout de chaque route qui n'est pas une sortie. Les intersections et les routes possèdent une stratégie, déterminant leur comportement sur le réseau.

### 1. Définition d'une route

Pour rester le plus simple possible, nous modélisons une route sous la forme d'un vecteur. Chaque case du vecteur représente une position qui ne peut-être occupée que par un véhicule à la fois. Au bout de chaque route se trouve une intersection, qu'il faudra ajouter manuellement après l'avoir définie. Nous implémentons également une fonction `est_feu_vert` qui indique si cette route à le droit de passer à l'intersection , s'il y en a une.

In [2]:
# Définition d'une route

class Route:

    def __init__(self, nom, type_route, longueur):
        self.nom = nom
        self.type_route = type_route # entree, sortie, lien
        self.longueur = longueur
        self.intersection = None
        self.vecteur = [0 for i in range(longueur)]

    def est_feu_vert(self):
        if self.type_route == "sortie":
            return True
        index = self.intersection.entrees.index(self)
        return True if self.intersection.phase_actuelle[index] == 1 else False

### 2. Définition d'une intersection

Une intersection est le croisement entre plusieurs routes. Pour éviter les colisions entre véhicules, celles-ci sont gérées par des feux bicolores, rouge ou vert. Une intersection possède un jeu de phases, qui représentent les droits de passage de l'intersection pour chacune des voies d'entrée. Une phase est représentée sous la forme d'un vecteur composés de 0 et de 1, indiquant respectivement si les véhicules d'une voie d'entrée ont feu rouge ou feu vert (l'ordre du vecteur d'une phase doit suivre le même ordre que celui des entrées). Chaque intersection possède une stratégie pour la sélection des phases : à chaque pas de simulation, l'intersection choisit de conserver ou de changer de phases selon cette stratégie. A cette fin, nous développons une méthode `changer_phase` qui sera invoquée lors de chaque tour de parole.

In [3]:
# Définition d'une intersection

class Intersection:

    def __init__(self, nom, entrees, sorties, phases, strategie):
        self.nom = nom
        self.entrees = entrees
        self.sorties = sorties
        self.phases = phases
        self.strategie = strategie
        self.phase_actuelle = phases[0]
        self.phase_actuelle_index = 0
        self.temps_phase_actuelle = 0

    def changer_phase(self):
        ancienne_phase = self.phase_actuelle
        self.temps_phase_actuelle += 1
        self.phase_actuelle = self.phases[self.strategie.prochaine_phase(self)]
        if self.phase_actuelle != ancienne_phase:
            self.temps_phase_actuelle = 0 
            self.phase_actuelle_index = self.phases.index(self.phase_actuelle)
            print("L'intersection {} passe à la phase {}".format(self.nom, self.phase_actuelle))

### 3. Définition d'une voiture

Les voitures sont les véhicules de base de notre SMA : ils circulent sur les routes, empruntent les intersections jusqu'à atteindre une destination finale, et se désactiver. Une voiture n'a qu'une seule méthode : `avancer`, qui fait avancer le véhicule le long de sa route **si c'est possible**. Si un autre véhicule est situé juste devant lui, il ne peut avancer et reste sur place. Lorsqu'il est situé sur la dernière case d'une route, le véhicule a deux options :
- si la route est une sortie, il se désactive et son trajet est terminé.
- sinon, il emprunte une nouvelle route à l'intersection (si le feu est vert) selon la stratégie qui lui est passée en paramètre.

In [4]:
# Définition d'un véhicule

class Voiture:

    def __init__(self, nom, route, strategie):
        self.nom = nom
        self.actif = True
        self.temps_actif = 0
        self.position_route = 0
        self.route_actuelle = route
        self.strategie = strategie

    def avancer(self):
        self.temps_actif += 1
        # Cas où le véhicule est en dernière position de la route
        if self.position_route == self.route_actuelle.longueur - 1:
            # Si la route est une sortie, alors le véhicule devient inactif
            if self.route_actuelle.type_route == "sortie":
                self.route_actuelle.vecteur[self.position_route] = 0
                self.actif = False
                print("Le véhicule {} a terminé son trajet en {} ticks.".format(self.nom, self.temps_actif))
            # Sinon, il choisit la route suivante selon sa stratégie, si le feu est vert et s'il n'y a personne sur la route suivante
            else:
                if self.route_actuelle.est_feu_vert():
                    prochaine_route = self.strategie.prochaine_route(self)
                    if prochaine_route.vecteur[0] == 0:
                        self.route_actuelle.vecteur[self.position_route] = 0
                        self.route_actuelle = prochaine_route
                        self.route_actuelle.vecteur[0] = self
                        self.position_route = 0
                        print("Le véhicule {} est passé sur la route {}.".format(self.nom, self.route_actuelle.nom))
        # S'il n'est pas au bout d'une voie, il avance s'il peut.
        else:
            if self.route_actuelle.vecteur[self.position_route + 1] == 0:
                self.position_route += 1
                self.route_actuelle.vecteur[self.position_route - 1] = 0
                self.route_actuelle.vecteur[self.position_route] = self         

### 4. Définition du *scheduler*

Le *scheduler* est la classe qui contrôle le système multi-agents. Attention lorsque l'on parle de contrôle : il ne s'agit pas ici de prendre le contrôle de chacun des agents, mais d'ordonner les temps de parole entre les agents. Les agents restent indépendants. Ici, nous nommons le *scheduler* `GestionnaireReseau`.

A chaque pas de simulation, le `GestionnaireReseau` donne la parole à toutes les intersections, puis à tous les véhicules, et génère enfin un véhicule sur la première case de chaque entrée selon une probabilité passée en paramètre. La méthode `run` exécute le SMA pendant $n$ ticks.

In [5]:
# Définition du gestionnaire réseau (Scheduler du SMA)

class GestionnaireReseau:

    def __init__(self, routes, intersections, proba, strategie_vehicule):
        self.routes = routes
        self.intersections = intersections
        self.proba = proba
        self.strategie_vehicules = strategie_vehicule
        self.vehicules = []
        self.compte_vehicules = 0

    def au_tour_des_intersections(self):
        random.shuffle(self.intersections)
        for intersection in self.intersections:
            intersection.changer_phase()

    def au_tour_des_vehicules(self):
        random.shuffle(self.vehicules)
        for vehicule in self.vehicules:
            if vehicule.actif:
                vehicule.avancer()

    def generer_vehicules(self):
        for route in routes:
            if route.type_route == "entree":
                if route.vecteur[0] == 0:
                    alea = random.random()
                    if alea <= self.proba:
                        self.compte_vehicules += 1
                        vehicule = Voiture(
                            nom = str(self.compte_vehicules),
                            route = route,
                            strategie = self.strategie_vehicules
                        )
                        route.vecteur[0] = vehicule
                        self.vehicules.append(vehicule)

    def run(self, nb_ticks):
        for _ in range(nb_ticks):
            self.au_tour_des_intersections()
            self.au_tour_des_vehicules()
            self.generer_vehicules()
        print("\n\nSIMULATION TERMINEE")

### 5. Stratégies

Ici, nous allons implémenter les stratégies pour les intersections et les véhicules.

#### a. Stratégie temps fixe

Pour les intersections, nous implémentons une simple stratégie à temps fixe : toutes les phases de l'intersection ont la même durée. Lorsque le temps de la phase est atteint, on passe à la phase suivante.

In [6]:
# Stratégie Intersection Temps Fixe

class TempsFixe:

    def __init__(self, temps_phase):
        self.temps_phase = temps_phase

    def prochaine_phase(self, intersection):
        if intersection.temps_phase_actuelle >= self.temps_phase:
            return (intersection.phase_actuelle_index + 1) % len(intersection.phases)
        else:
            return intersection.phase_actuelle_index

#### b. Stratégie aléatoire uniforme

Pour les véhicules, nous implémentons une stratégie aléatoire uniforme. Lorsque le véhicule atteint une intersection, il choisit aléatoirement une sortie parmi celles possibles, avec la même probabilité pour chaque.

In [7]:
# Stratégie Véhicule Aléatoire Uniforme

class DirectionAleatoireUniforme:

    def prochaine_route(self, vehicule):
        sorties_possibles = vehicule.route_actuelle.intersection.sorties
        return random.choice(sorties_possibles)

### 6. Instanciation du réseau

Nous créons ici, à l'aide des composants que nous avons développés plus haut, un réseau grille 2x2, c'est à dire un réseau composé de 2x2 intersections. Chaque route fait 50 cases de long (les intersections n'ont pas de longueur), et chaque véhicule et intersection est implémenté avec la stratégie qui leur a été développée plus haut.

In [10]:
# Créer le réseau grille 2x2

# Définition des routes
s1_A = Route('s1_A', 'entree', 50); s8_A = Route('s8_A', 'entree', 50); D_A = Route('D_A', 'lien', 50); B_A = Route('B_A', 'lien', 50)
A_s1 = Route('A_s1', 'sortie', 50); A_s8 = Route('A_s8', 'sortie', 50); A_D = Route('A_D', 'lien', 50); A_B = Route('A_B', 'lien', 50)
s2_B = Route('s2_B', 'entree', 50); s3_B = Route('s3_B', 'entree', 50); C_B = Route('A_D', 'lien', 50)
B_s2 = Route('B_s2', 'sortie', 50); B_s3 = Route('B_s3', 'sortie', 50); B_C = Route('B_C', 'lien', 50)
s4_C = Route('s4_C', 'entree', 50); s5_C = Route('s5_C', 'entree', 50); D_C = Route('D_C', 'lien', 50)
C_s4 = Route('C_s4', 'sortie', 50); C_s5 = Route('C_s5', 'sortie', 50); C_D = Route('C_D', 'lien', 50)
s6_D = Route('s6_D', 'entree', 50); s7_D = Route('s7_D', 'entree', 50);
D_s6 = Route('D_s6', 'sortie', 50); D_s7 = Route('D_s7', 'sortie', 50);
routes = [s1_A, s8_A, D_A, B_A, A_s1, A_s8, A_D, A_B, s2_B, s3_B, C_B, B_s2, B_s3, B_C, s4_C, s5_C, D_C, C_s4, C_s5, C_D, s6_D, s7_D, D_s6, D_s7]


# Définition des stratégies
strategie_vehicules = DirectionAleatoireUniforme()
strategie_intersections = TempsFixe(30)


# Définition des intersections
A = Intersection(
    nom = 'A',
    entrees = [s1_A, s8_A, D_A, B_A],
    sorties = [A_s1, A_s8, A_D, A_B],
    phases = [[0, 1, 0, 1], [1, 0, 1, 0]],
    strategie = strategie_intersections
)

B = Intersection(
    nom = 'B',
    entrees = [s2_B, s3_B, A_B, C_B],
    sorties = [B_s2, B_s3, B_A, B_C],
    phases = [[0, 1, 0, 1], [1, 0, 1, 0]],
    strategie = strategie_intersections
)

C = Intersection(
    nom = 'C',
    entrees = [B_C, s4_C, s5_C, D_C],
    sorties = [C_B, C_s4, C_s5, C_D],
    phases = [[0, 1, 0, 1], [1, 0, 1, 0]],
    strategie = strategie_intersections
)

D = Intersection(
    nom = 'D',
    entrees = [A_D, C_D, s6_D, s7_D],
    sorties = [D_A, D_C, D_s6, D_s7],
    phases = [[0, 1, 0, 1], [1, 0, 1, 0]],
    strategie = strategie_intersections
)

intersections = [A, B, C, D]

# Ajout manuel des intersections aux routes
s1_A.intersection = A; s8_A.intersection = A; D_A.intersection = A; B_A.intersection = A
A_B.intersection = B; s2_B.intersection = B; s3_B.intersection = B; C_B.intersection = B
B_C.intersection = C; s4_C.intersection = C; s5_C.intersection = C; D_C.intersection = C
C_D.intersection = D; s6_D.intersection = D; s7_D.intersection = D; A_D.intersection = D; 

Et nous lançons le SMA sur 1000 pas de simulation.

In [11]:
# Définition du gestionnaire

gest = GestionnaireReseau(
    routes = routes,
    intersections = intersections,
    proba = 0.2,
    strategie_vehicule = strategie_vehicules
)

gest.run(1000)

L'intersection B passe à la phase [1, 0, 1, 0]
L'intersection A passe à la phase [1, 0, 1, 0]
L'intersection D passe à la phase [1, 0, 1, 0]
L'intersection C passe à la phase [1, 0, 1, 0]
Le véhicule 2 est passé sur la route B_C.
Le véhicule 3 est passé sur la route A_s1.
Le véhicule 6 est passé sur la route D_C.
Le véhicule 7 est passé sur la route A_s1.
Le véhicule 10 est passé sur la route A_B.
L'intersection C passe à la phase [0, 1, 0, 1]
L'intersection D passe à la phase [0, 1, 0, 1]
L'intersection B passe à la phase [0, 1, 0, 1]
L'intersection A passe à la phase [0, 1, 0, 1]
Le véhicule 5 est passé sur la route C_s4.
Le véhicule 1 est passé sur la route B_s3.
Le véhicule 9 est passé sur la route D_s6.
Le véhicule 12 est passé sur la route A_D.
Le véhicule 13 est passé sur la route A_D.
Le véhicule 4 est passé sur la route B_C.
Le véhicule 8 est passé sur la route B_A.
Le véhicule 17 est passé sur la route A_B.
Le véhicule 22 est passé sur la route B_s3.
Le véhicule 25 est passé 

# B. Exercices

### I. Comportements de véhicules
- Créez une stratégie qui donne une destination à atteindre à la voiture. La voiture doit prendre le plus court chemin pour arriver à sa destination. La classe doit implémenter une fonction `prochaine_route`.
- Ajoutez une classe Camion qui représente un camion dans le système. Un camion fait deux cases de long, et lorsqu'il est à l'arrêt, il ne redémarre qu'au deuxième tick après que la voie se soit libérée devant lui. La classe doit forcément posséder une fonction `avancer`.

### II. Comportements d'intersections
Implémentez les comportements d'intersection suivants (toutes les classes doivent implémenter une méthode `prochaine_phase`) :
- Longest Queue First : La prochaine phase sélectionnée est celle qui a la plus grande file de véhicules depuis l'intersection. Une file est une suite de case du vecteur de la route qui sont occupées par un véhicule, depuis le feu.
- Feu booléen : L'intersection possède des capteurs sur chaque voie qui lui permettent de repérer si un véhicule est en attente sur une portée de 5 cases. Si la phase active ne possède pas de véhicules à faire passer sur cette distance, alors on passe à la phase suivante. Si le temps de la phase actuelle dépasse un temps maximum à passer en paramètre, alors on passe à la phase suivante. 
- Self Organizing Traffic Lights : Même comportement que le feu booléens, sauf qu'on ajoute un seuil comme paramètre. Lorsque le temps d'attente cumulé des véhicules qui sont sur des voies pour laquelle le feu est rouge dépasse ce seuil, on passe à la phase suivante.
- Max Pressure : Calculez la pression pour chaque phase. La pression est égale à la différence entre le nombre de véhicules sur les voies vertes de la phase, et le nombre de véhicules en sortie. La phase qui maximise la pression est sélectionnée.
- **Bonus** Coordination : (A ajouter au code des autres comportements) Lorsque qu'un certain nombre de véhicules (passé en paramètre) traverse l'intersection vers la même route, l'intersection suivante est obligée de passer verte pour cette voie si ce n'est pas déjà le cas.

### III. Comportement du SMA
Jusqu'à maintenant, nous sélectionnions aléatoirement l'ordre des tours de parole des véhicules dans le `GestionnaireReseau`. Cela entraîne des problèmes : les véhicules devraient pouvoir avancer en même temps et ce n'est pas le cas ici, ce qui n'est pas réaliste. Modifiez cette classe pour faire en sorte que tous les véhicules se déplacent "en même temps" pendant un tour de paroles.

*Note : La solution à adopter consiste à créer un réseau "jumeau" temporaire pour simuler le déplacement de tous les véhicules, et à résoudre les conflits ensuite.*

### IV. Etude de performance
Réalisez une étude des performance de chacun des comportements d'intersection, en réalisant plusieurs simulations et en faisant varier les paramètres (attention : les différents comportements doivent être évalués sur les mêmes paramètres). L'objectif est de minimiser le temps de trajet moyen des utilisateurs. Lequel est le meilleur ?